In [1]:
import subprocess
import numpy as np
import mdtraj as md
from tqdm import tqdm_notebook
import time
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter as sg
%pylab inline

print "RMSD - Fraction of Native Contacts (alpha helix) minus Fraction of Non-Native Contacts\n"
print "‘H’ : Helix       ‘G’ : 3-helix (3/10 helix)\n'E’ : Beta ladder ‘B’ : Beta-bridge\n‘C’ : Random Coil ‘T’ : Hydrogen bonded turn\n"
gro_reference = '/home/karl/VoelzLab/ProteinG/gro/p8600/protein_only.gro'
traj_references = '/media/karl/VAV6/PROJ8600/RUN%d/CLONE%d/frame%d.xtc'
nruns=150
clone=2
trajs = [[] for i in range(nruns)]

#loads trajectories for 1 clone in every run in project.
def load_runs():
    for run in tqdm_notebook(range(nruns), desc='Loading Trajs.'):
        time.sleep(0.01)
        frame = int(subprocess.check_output('ls /media/karl/VAV6/PROJ8600/RUN%d/CLONE%d/frame*.xtc | wc -l' % (run,clone),shell=True))
        if frame >= 300: #if you have a lot of runs, try reducing amount by only selecting ones with long traj lengths
            for frm in range(frame):
                loadtrajs = md.load(traj_references%(run,clone,frm),top=gro_reference)
                trajs[run].append(loadtrajs)
load_runs()
#np.save('trajs',trajs) to save them to disk

Populating the interactive namespace from numpy and matplotlib
RMSD - Fraction of Native Contacts (alpha helix) minus Fraction of Non-Native Contacts

‘H’ : Helix       ‘G’ : 3-helix (3/10 helix)
'E’ : Beta ladder ‘B’ : Beta-bridge
‘C’ : Random Coil ‘T’ : Hydrogen bonded turn




In [13]:
trajs2 = [t for t in trajs if t!=[]]
trajs3 = [t for t in trajs2 if t[-1].time[-1]>0]

In [7]:
dssp = [[] for i in range(len(trajs3))]
def calculate_dssp():
    for run in tqdm_notebook(range(len(trajs3)), desc='DSSP Progress'):
        for traj in range(len(trajs3[run])):
            dssp[run].append(md.compute_dssp(trajs3[run][traj]))
            time.sleep(0.01)
    #np.save('dssp',dssp)
calculate_dssp()

In [8]:
print len(dssp)
print len(dssp[0])

4
348


In [19]:
helix=[[] for i in range(len(dssp))]
betasheet=[[] for i in range(len(dssp))]
rmsds = [[] for i in range(len(dssp))]

#Gets DSSP classification from md.compute_dssp, sums up individual codes(eg H,C,B..etc) per frame, appends them to
#individual lists with lengths = total trajectory length
for run in range(len(dssp)):
    for traj in range(len(dssp[run])):
        for ts in range(len(dssp[run][traj])):
            RMSDreference = trajs3[run][0]
            rmsds[run].append(md.rmsd(trajs3[run][traj],reference=md.load(gro_reference)))
            helixsum=0
            betasum=0
            for code in range(len(dssp[run][traj][ts])): # for i in range length 3rd dimension
                if dssp[run][traj][ts][code] == 'H' or dssp[run][traj][ts][code] == 'I' or dssp[run][traj][ts][code] == 'G':
                    helixsum+=1
                elif dssp[run][traj][ts][code] == 'E' or dssp[run][traj][ts][code] == 'B':
                    betasum+=1
            helix[run].append(helixsum)
            betasheet[run].append(betasum)

KeyboardInterrupt: 

In [18]:
print len(rmsds)
print rmsds[0]

4


KeyboardInterrupt: 